# Homework 4: Sentence Segmentation

Sentence segmentation is a crucial, yet under-appreciated, part of any text processing pipeline. In any real-world setting, text will not come to you pre-chunked into sentences, and if your pipeline involves any sentence-level analysis (parsing, translation, entity & co-reference extraction, etc.) the first thing you'll need to do will be split things up into sentences. This is a deceptively challenging problem, as punctuation can be used in multiple ways. Consider the following sentence (borrowed from Kyle Gorman's [excellent discussion of the subject](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/)): 

> _"Rolls-Royce Motor Cars Inc. said it expects its U.S. sales to remain steady at about 1,200 cars in 1990."_ 

This sentence features several periods, but only one of them (the last one) represents a valid sentence break. Sentences can also have embedded clauses or quotation marks, some of which may superficially look like a sentence boundary: 

> _He says the big questions–“Do you really need this much money to put up these investments? Have you told investors what is happening in your sector? What about your track record?–“aren’t asked of companies coming to market._

There are many ways to approach this particular problem; it can be approached using rules, which themselves may be encoded as regular expressions, or (more usefully) it can be modeled as a machine learning problem. Specifically, as a binary classification problem, where candidate sentence boundaries (i.e., punctuation marks from the closed class of English punctuation that can represent a sentence boundary) are either _positive_ examples (actual sentence boundaries) or _false_ (not sentence boundaries). 

In this assignment, you will develop and test your own sentence boundary detection algorithm. This assignment is shorter than our previous assignments, and is also much more open-ended. By this point in the class, you have been exposed to several different families of technique for text classification and analysis, and have hopefully begun to build up some intuitions. For this assignment, you can choose whatever methods you like: log-linear models, rule-based methods, some sort of neural network, it's up to you!

## 0. Setup & data exploration

In [ ]:
from importlib import reload
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from hw4_utils import data
from hw4_utils import classifier

We've provided a version of the WSJ section of the Penn Treebank to use for training and evaluation purposes. See `data/wsj_sbd/README.md` for a description of where this data came from and how it was assembled.

The `hw4_utils.data` module has a useful method for reading in data (`file_as_string`) and another utility function to extract candidate boundaries. Let's use these to explore our data a little bit.

In [3]:
dev = data.file_as_string("data/wsj_sbd/dev.txt")
train = data.file_as_string("data/wsj_sbd/train.txt")
test = data.file_as_string("data/wsj_sbd/test.txt")

In [4]:
dev[:100]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is c'

In [5]:
dev[:300]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is chairman of Elsevier N.V., the Dutch publishing group.\nRudolph Agnew, 55 years old and former chairman of Consolidated Gold Fields PLC, was named a nonexecutive director of this British industrial cong'

Each of these is a string containing the text, with one input sentence per line. Note that newline characters are still present, which will be important as this will provide us with our ground truth.

Remember, we _only_ will use the `test` split for our final evaluation. For data exploration, feature engineering, and for tuning hyperparameters, use the `dev` split.

The first and most important thing to find out when doing classification is what your class probabilities are, so let's look into that. The `load_candidates()` function in the `hw4_utils.data` module will read an input data split, identify candidate boundaries, and extract information about the candidates context that we may want to use for feature engineering. One of the attributes it will extract is whether or not the candidate is actually a true sentence break (`is_true_break`). Obviously, this would only be useful at training time, as in the "real world" we wouldn't know the answer. :-)

We will use `load_candidates()` to answer our question about class balance:

load_candidates() identifies _potential_ candidate boundaries

In [6]:
list(data.load_candidates(dev))

[Observation(left_token='Nov', left_raw='d as a nonexecutive director Nov', punctuation_mark='.', right_token='29', right_raw='29. Mr. Vinken is chairman of El', is_true_break=False, end_offset=81, orig_obs='d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El'),
 Observation(left_token='29', left_raw=' a nonexecutive director Nov. 29', punctuation_mark='.', right_token='Mr', right_raw='Mr. Vinken is chairman of Elsevi', is_true_break=True, end_offset=85, orig_obs=' a nonexecutive director Nov. 29. Mr. Vinken is chairman of Elsevi'),
 Observation(left_token='Mr', left_raw='onexecutive director Nov. 29. Mr', punctuation_mark='.', right_token='Vinken', right_raw='Vinken is chairman of Elsevier N', is_true_break=False, end_offset=89, orig_obs='onexecutive director Nov. 29. Mr. Vinken is chairman of Elsevier N'),
 Observation(left_token='group', left_raw='N.V., the Dutch publishing group', punctuation_mark='.', right_token='Rudolph', right_raw='Rudolph Agnew, 55 years old and

In [7]:
from collections import Counter

Counter([c.is_true_break for c in data.load_candidates(dev)])

Counter({False: 1766, True: 5769})

As is often the case, our classes are quite imbalanced! Let's load in our ground truth labels:

In [8]:
y_dev = [o.is_true_break for o in data.load_candidates(dev)]

What else can we find out about our candidate breaks?

In [9]:
candidates = list(data.load_candidates(dev[:10000])) # for now, just get candidates from the first 10k characters
first_can = candidates[0]
first_can

Observation(left_token='Nov', left_raw='d as a nonexecutive director Nov', punctuation_mark='.', right_token='29', right_raw='29. Mr. Vinken is chairman of El', is_true_break=False, end_offset=81, orig_obs='d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El')

In [10]:
print("original context: ",first_can.orig_obs)
print("punctuation mark: ", first_can.punctuation_mark)
print("token to the left: ", first_can.left_token)
print("token to the right: ", first_can.right_token)

original context:  d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El
punctuation mark:  .
token to the left:  Nov
token to the right:  29


Take a look at the the `Observation` namedtuple in `hw4_utils.data` for more details about the various attributes that you can find out about each possible sentence boundary.

## 1. Baselines

We will begin by looking at three different baselines. First, we will 

### Baseline #1: Majority class

Since our data are quite imbalanced, what if we always treat every candidate boundary as though it was a true boundary?

In [11]:
y_hat_baseline = [classifier.baseline_classifier(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_baseline))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00      1766
        True       0.77      1.00      0.87      5769

   micro avg       0.77      0.77      0.77      7535
   macro avg       0.38      0.50      0.43      7535
weighted avg       0.59      0.77      0.66      7535



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Baseline #2: Next-token capitalization

What if we say that a candidate is a boundary if the following token is capitalized?

In [12]:
y_hat_next_tok_cap = [classifier.next_tok_capitalized_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_next_tok_cap))

              precision    recall  f1-score   support

       False       0.60      0.40      0.48      1766
        True       0.83      0.92      0.87      5769

   micro avg       0.80      0.80      0.80      7535
   macro avg       0.71      0.66      0.67      7535
weighted avg       0.78      0.80      0.78      7535



### Baseline #3: Punkt

Using the [NLTK implementation](http://www.nltk.org/_modules/nltk/tokenize/punkt.html) of [Punkt (Kiss & Strunk, 2006)](https://www.mitpressjournals.org/doi/pdf/10.1162/coli.2006.32.4.485):

(Note: You'll need to have downloaded the NLTK pre-trained Punkt model for this baseline to work)

In [13]:
y_hat_punkt = [classifier.punkt_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_punkt))

              precision    recall  f1-score   support

       False       0.92      0.43      0.59      1766
        True       0.85      0.99      0.91      5769

   micro avg       0.86      0.86      0.86      7535
   macro avg       0.88      0.71      0.75      7535
weighted avg       0.87      0.86      0.84      7535



### Conclusion

None of these are especially inspiring in their performance... can you do better?

## 2. Your turn!

### Part 1: Now it's your turn.

Your mission, in this assignment, is to implement at least two additional approaches. The approaches must be different in terms of the features you choose to use, classification approach, or both, but beyond that limitation you are free to do whatever you like as long as it is your own work (i.e., do not download and use an already-existing sentence boundary detector tool). You may, however, replicate an existing algorithm, though please be very careful not about re-using other people's code. 

For a discussion of features that may prove useful, and of other approaches to this task that you may wish to replicate or build off of, consult the bibliography discussed in [Gorman (2014)](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/); note that the specific sentence boundary detector described on that page uses a small set of highly useful features, but you will want to go beyond simply replicating his system.

***Deliverable***: The code for your ≥ two additional classifiers, along with a brief writeup (≈1 page) of what you built and why you chose those specific approaches and features. 

### Part 2: Evaluation & Error analysis

After building and describing your additional classifiers, evaluate their performance against the `test` partition of the data as shown above. Write a brief summary of their relative performance, and include an _error analysis_. Examine cases where your classifiers failed to predict correctly, or disagreed with one another, and see if you can identify any patterns that might explain where they were going awry.

***Deliverable***: Your written summary, as well as any relevant data tables.

**Logistic Regression**

In [14]:
# loading in libraries
import torch
import torch.nn as nn
import string
import random
import torch.nn.functional as F
import numpy as np

In [15]:
# loading in the training candidates
train_candidates = list(data.load_candidates(train))

In [16]:
# here, I can print out training examples for observation--note that I never do this with the test set.
train_candidates_short = train_candidates
random.shuffle(train_candidates_short)
train_candidates_short = train_candidates_short[:100]
i = 0
for ex in train_candidates_short:
    print(str(i) + ":")
    print(ex)
    i+=1

0:
Observation(left_token='opening', left_raw=' But a few new spots are opening', punctuation_mark='.', right_token='Arthur', right_raw='Arthur Andersen, the accounting ', is_true_break=True, end_offset=45331, orig_obs=' But a few new spots are opening. Arthur Andersen, the accounting ')
1:
Observation(left_token='dollars', left_raw='ween one and two billion dollars', punctuation_mark='.', right_token='So', right_raw='So you get an idea of the levera', is_true_break=True, end_offset=252615, orig_obs='ween one and two billion dollars. So you get an idea of the levera')
2:
Observation(left_token='hammered', left_raw='day after the stock got hammered', punctuation_mark='.', right_token='If', right_raw='If the market drops further toda', is_true_break=True, end_offset=3972931, orig_obs='day after the stock got hammered. If the market drops further toda')
3:
Observation(left_token='report', left_raw='n Lehman Hutton Inc. in a report', punctuation_mark='.', right_token='This', right_raw='Thi

In [17]:
# my acronym list I created from observation and extension
acronyms_list = ["Inc", "St","Mr", "Mrs", "Ms", "Dr", "Jan", "Feb", "Aug", "Sept", "Oct", "Nov", "Dec", "U.S", "Corp", "Co", "Ltd", "Sen"]

In [18]:
# my functions for creating my x and y tensors
def make_large_tensor(candidates):
    y = []
    for i in range(0, len(candidates)):
        candidate = candidates[i]
        next_word = candidate.right_token
        prev_word = candidate.left_token
        #print(prev_word)
        case_of_next_word = next_word[0].isupper()
        case_of_prev_word = prev_word[0].islower()
        if case_of_next_word:
            x1 = 1
        else:
            x1 = 0
        if case_of_prev_word:
            x2 = 1
        else:
            x2 = 0
        if prev_word in acronyms_list:
            #print(prev_word)
            x3 = 0
        else:
            x3 = 1
        x = [x1, x2, x3]
        #x = [x1, x3]
        y.append(x)
    y = np.array(y)
    y = torch.from_numpy(y)
    return y

def make_y_tensor(candidates):
    vec = []
    for candidate in candidates:
        if candidate.is_true_break:
            vec.append([1])
        else:
            vec.append([0])
    vec = np.array(vec)
    vec = torch.from_numpy(vec)
    return vec

In [19]:
# where the actual logistic regression occurs
loss_func = nn.NLLLoss()

def eval_acc(y_hat_, y_):
    n_correct = (y_hat_.argmax(dim=1) == y_).sum().float()
    n_total = float(len(y_))
    return (n_correct / n_total).item()

class LogReg(nn.Module):
    def __init__(self, input_dims, output_classes):
        super(LogReg, self).__init__()
        self.linear = nn.Linear(input_dims, output_classes)
    
    def forward(self, x):
        x = x.double()
        z = self.linear(x)
        #print(z)
        z = z.double()
        return F.log_softmax(z, dim=1)



def train(model, num_epochs, training_data_x, training_data_y):
    sgd_opt = torch.optim.SGD(model.parameters(), lr=0.01)

    for i in range(num_epochs):
        sgd_opt.zero_grad() # erase any gradients our optimizer might remember

        # do a forward pass through our data:
        y_hat = model(training_data_x) 

        #print(y_hat[:20])
        #print(y_train.squeeze())

        loss = loss_func(y_hat, training_data_y.squeeze().long())
        #print(loss)

        loss.backward()
        sgd_opt.step()

        epoch_acc = eval_acc(y_hat, training_data_y.squeeze())

        #print out how we're doing:
        if i % 100 == 0:
            print(f"epoch {i}, training loss: {loss}, train acc: {eval_acc(y_hat, training_data_y.squeeze())}")
        if i == num_epochs-1:
            print(f"epoch {i}, training loss: {loss}, train acc: {eval_acc(y_hat, training_data_y.squeeze())}")
    return model

In [20]:
# testing my results and producing a list of True and False to compare to the ground truth
def prediction(model, sample):
    #print(result.squeeze(0))
    with torch.no_grad():
        result = model(sample.unsqueeze(0))
        result = result.squeeze(0)
        print(result)
        result = result.numpy()
        result = np.exp(result)
        #print(result)
        if result[1] > result[0]:
            return True
        else:
            return False
        
def test_predictions(model, whole_test_sample):
    results = []
    for sample in whole_test_sample:
        print(sample)
        results.append(prediction(model, sample))
    return results

In [21]:
# creating my x training data
x_train = make_large_tensor(train_candidates)

In [22]:
x_train.size()

torch.Size([44279, 3])

In [23]:
x_train[:20]

tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [0, 0, 0],
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
        [1, 0, 0],
        [0, 0, 0],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 1],
        [1, 0, 0],
        [1, 1, 1]])

In [24]:
#creating my y training data
y_train = make_y_tensor(train_candidates)

In [25]:
y_train.squeeze()[:100]

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1])

In [35]:
# training the model with x_train and y_train
trained_model = train(
    model = LogReg(3,2).double(), 
    num_epochs = 1000,
    training_data_x= x_train,
    training_data_y = y_train)

epoch 0, training loss: 1.0941402843688788, train acc: 0.21348720788955688
epoch 100, training loss: 0.40526334483528353, train acc: 0.8622823357582092
epoch 200, training loss: 0.33543045488895507, train acc: 0.8622823357582092
epoch 300, training loss: 0.3118161585483954, train acc: 0.8622823357582092
epoch 400, training loss: 0.29683100604632645, train acc: 0.8622823357582092
epoch 500, training loss: 0.284854074274163, train acc: 0.8622823357582092
epoch 600, training loss: 0.2745564884660991, train acc: 0.8622823357582092
epoch 700, training loss: 0.26549005507382856, train acc: 0.8622823357582092
epoch 800, training loss: 0.25743575867489626, train acc: 0.9439011812210083
epoch 900, training loss: 0.2502485069695714, train acc: 0.9439011812210083
epoch 999, training loss: 0.2438757340118428, train acc: 0.9439011812210083


In [36]:
# loading in the test candidates
test_candidates = list(data.load_candidates(test))

# creating the test tensors
x_test = make_large_tensor(test_candidates)

In [52]:
# predicting the results of test, by running it through my model
test_results = test_predictions(trained_model, x_test)

tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1,

tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1,

tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([0, 0, 1])
tensor([-1.4190, -0.2770], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2

tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1,

tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 1])
tensor([-1.4190, -0.2770], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0

...

tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2

tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 1])
tensor([-1.4190, -0.2770], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2

tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1,

tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([0, 1, 1])
tensor([-2.3887, -0.0962], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2

tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 0])
tensor([-0.5936, -0.8037], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([0, 0, 0])
tensor([-0.4114, -1.0869], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 0, 1])
tensor([-1.7900, -0.1827], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2.8194, -0.0615], dtype=torch.float64)
tensor([1, 1, 1])
tensor([-2

In [38]:
len(test_results)

9736

In [39]:
# creating my ground truth comparison for the test set
y_test = [o.is_true_break for o in data.load_candidates(test)]

In [40]:
len(y_test)

9736

In [41]:
# printing out the classification results for my predicted results
print(classification_report(y_test, test_results))

              precision    recall  f1-score   support

       False       0.93      0.83      0.88      2205
        True       0.95      0.98      0.97      7531

   micro avg       0.95      0.95      0.95      9736
   macro avg       0.94      0.91      0.92      9736
weighted avg       0.95      0.95      0.95      9736



In [53]:
for param in trained_model.parameters():
    print(param)

Parameter containing:
tensor([[-0.7041, -0.6056, -1.2491],
        [-0.2387,  0.5450,  0.5684]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([ 0.1991, -0.4764], dtype=torch.float64, requires_grad=True)


**Rule Based**

In [43]:
def is_sentence_boundary(examples):
    results = []
    for example in examples:
        next_word = example.right_token
        prev_word = example.left_token
        case_of_next_word = next_word[0].isupper()
        case_of_prev_word = prev_word[0].islower()
        if prev_word in acronyms_list:
            prev_word_acronym = True
        else:
            prev_word_acronym = False

            
        if prev_word_acronym:
                results.append(False)
        else:
            if case_of_next_word or case_of_prev_word:
                results.append(True)
            else:
                results.append(False)
    return results

In [44]:
rule_based_results = is_sentence_boundary(data.load_candidates(test))
rule_based_results

[True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 Tru

In [45]:
print(classification_report(y_test,rule_based_results))

              precision    recall  f1-score   support

       False       0.90      0.87      0.88      2205
        True       0.96      0.97      0.97      7531

   micro avg       0.95      0.95      0.95      9736
   macro avg       0.93      0.92      0.93      9736
weighted avg       0.95      0.95      0.95      9736



In [46]:
len(test_results)

9736

In [47]:
len(rule_based_results)

9736

In [48]:
#this I should only be doing on test if I don't plan to change the algorithm
def compare_predictions(candidates, ground_truth, my_predictions):
    count = 0
    i = 0
    while i < len(my_predictions):
        if ground_truth[i]!= my_predictions[i]:
            print("My prediction: " + str(my_predictions[i]))
            print("Actual result: " + str(ground_truth[i]))
            print(candidates[i])
            count+=1
        i+=1
    print("Number incorrectly classified: " + str(count))
    print("Total number examples: " + str(len(my_predictions)))
    print("Percentage correctly classified: " + str((len(my_predictions) - count)/len(my_predictions)))

In [49]:
compare_predictions(list(data.load_candidates(test)), y_test, rule_based_results)

My prediction: False
Actual result: True
Observation(left_token='Salinas', left_raw=' weigh so heavily on Mr. Salinas', punctuation_mark='.', right_token='``', right_raw='"The Japanese will come to Mexic', is_true_break=True, end_offset=2640, orig_obs=' weigh so heavily on Mr. Salinas. "The Japanese will come to Mexic')
My prediction: True
Actual result: False
Observation(left_token='W', left_raw='needed to be done. When Thomas W', punctuation_mark='.', right_token='Wathen', right_raw='Wathen went big league last year', is_true_break=False, end_offset=18077, orig_obs='needed to be done. When Thomas W. Wathen went big league last year')
My prediction: False
Actual result: True
Observation(left_token='Inc', left_raw="curity business: Pinkerton's Inc", punctuation_mark='.', right_token='There', right_raw='There was a wanted poster offeri', is_true_break=True, end_offset=18224, orig_obs="curity business: Pinkerton's Inc. There was a wanted poster offeri")
My prediction: True
Actual result:

My prediction: False
Actual result: True
Observation(left_token='Inc', left_raw='ares, via Alex. Brown & Sons Inc', punctuation_mark='.', right_token='Harmonia', right_raw='Harmonia Bancorp Inc. -- 4,750,0', is_true_break=True, end_offset=924148, orig_obs='ares, via Alex. Brown & Sons Inc. Harmonia Bancorp Inc. -- 4,750,0')
My prediction: False
Actual result: True
Observation(left_token='Inc', left_raw=', via Shearson Lehman Hutton Inc', punctuation_mark='.', right_token='Healthsource', right_raw='Healthsource Inc. -- Two million', is_true_break=True, end_offset=924230, orig_obs=', via Shearson Lehman Hutton Inc. Healthsource Inc. -- Two million')
My prediction: False
Actual result: True
Observation(left_token='Co', left_raw='shares, via Kidder, Peabody & Co', punctuation_mark='.', right_token='Immune', right_raw='Immune Response Corp. -- Three m', is_true_break=True, end_offset=924304, orig_obs='shares, via Kidder, Peabody & Co. Immune Response Corp. -- Three m')
My prediction: False


In [50]:
compare_predictions(list(data.load_candidates(test)), y_test, test_results)

My prediction: True
Actual result: False
Observation(left_token='W', left_raw='needed to be done. When Thomas W', punctuation_mark='.', right_token='Wathen', right_raw='Wathen went big league last year', is_true_break=False, end_offset=18077, orig_obs='needed to be done. When Thomas W. Wathen went big league last year')
My prediction: False
Actual result: True
Observation(left_token='Inc', left_raw="curity business: Pinkerton's Inc", punctuation_mark='.', right_token='There', right_raw='There was a wanted poster offeri', is_true_break=True, end_offset=18224, orig_obs="curity business: Pinkerton's Inc. There was a wanted poster offeri")
My prediction: True
Actual result: False
Observation(left_token='W', left_raw=' Robbers Frank James and Jesse W', punctuation_mark='.', right_token='James', right_raw='James" and the original Pinkerto', is_true_break=False, end_offset=18337, orig_obs=' Robbers Frank James and Jesse W. James" and the original Pinkerto')
My prediction: False
Actual result: